In [1]:
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import nltk
from __future__ import division
import math
from sklearn.svm import LinearSVC
import string
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import os
#handle overfitting
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding,Dropout, Bidirectional

from keras.callbacks import EarlyStopping
from keras.layers import GlobalAveragePooling1D
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.initializers import Constant
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from keras.utils.np_utils import to_categorical
import re
from keras import regularizers
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('D:/downloads_14/H3_Multiclass_Hate_Speech_Detection_train.csv')
df.head(3)
df['label'].unique()

,label,tweet,id
0,1,"""@KeyshawnSwag: Lmfao this cat started beating...",0
1,1,RT @digaveliavelife: Lol I be eatin da shit ou...,1
2,1,RT @QueenReenie_: How bitch how? &#8220;@_Vont...,2


In [3]:
test = pd.read_csv("C:/Users/HP/Downloads/H3_Multiclass_Hate_Speech_Detection_test.csv") #,encoding='unicode_escape'

In [5]:
def clean_str(in_str):
    in_str = str(in_str)
    # replace urls with 'url'
    in_str = re.sub(r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})", "url", in_str)
    in_str = re.sub(r'([^\s\w]|_)+', '', in_str)
    return in_str.strip().lower()


df['tweet'] = df['tweet'].apply(clean_str)
test['tweet'] = test['tweet'].apply(clean_str)

In [6]:
df.label.value_counts()

1    15398
2     3291
0     1137
Name: label, dtype: int64

In [7]:
df_0 = df[df['label'] == 0].sample(frac=1)
df_1 = df[df['label'] == 1].sample(frac=1)
df_2 = df[df['label'] == 2].sample(frac=1)
sample_size = 1137
data = pd.concat([df_0.head(sample_size), df_1.head(sample_size), df_2.head(sample_size)]).sample(frac=1)

In [8]:
data['l'] = data['tweet'].apply(lambda x: len(str(x).split(' ')))
print("mean length of sentence: " + str(data.l.mean()))
print("max length of sentence: " + str(data.l.max()))
print("std dev length of sentence: " + str(data.l.std()))

mean length of sentence: 14.101436528877162
max length of sentence: 32
std dev length of sentence: 6.821066706531941


## LR with CountVectorizer

In [12]:
review = data['tweet'].values
label = data['label'].values
review_train, review_test, label_train, label_test = train_test_split(review, label, test_size=0.25, random_state=1000)

review_vectorizer = CountVectorizer()
review_vectorizer.fit(review_train)
Xlr_train = review_vectorizer.transform(review_train)
Xlr_test  = review_vectorizer.transform(review_test)

LRmodel = LogisticRegression()
LRmodel.fit(Xlr_train, label_train)
score = LRmodel.score(Xlr_test, label_test)
print("Accuracy:", score) 

## LR with Tf-idf

In [16]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(train.tweet).toarray()
labels = train.label
features.shape
LRmodel = LogisticRegression()
LRmodel.fit(features, labels)

# # score = LRmodel.score(Xlr_test, label_test)
# # print("Accuracy:", score) 
features_test = tfidf.transform(test.tweet)
features_test.shape
pred = LRmodel.predict(features_test)
submission=test
submission['label']=pred
# submission.drop(['', 'Phrase'], axis=1, inplace=True)
# submission.to_csv("submission.csv", index = False)
submission.to_csv("D:/res5.csv")

<4957x12318 sparse matrix of type '<class 'numpy.int64'>'
	with 64793 stored elements in Compressed Sparse Row format>

In [125]:
#NB

In [126]:
nb_tfidf = MultinomialNB()
nb_tfidf.fit(Xlr_train, label_train)  #model
y_predict = nb_tfidf.predict(Xlr_test)
print(classification_report(label_test,y_predict))

MultinomialNB()

In [ ]:
## Best parameters for NB

In [128]:
params = {'alpha': [0.01,0.1,0.5,1,10],
         }

multinomial_nb_grid = GridSearchCV(MultinomialNB(), param_grid=params, n_jobs=-1, cv=5, verbose=5)
multinomial_nb_grid.fit(Xlr_train, label_train)
y_predict = multinomial_nb_grid.predict(Xlr_test)
print('Best Accuracy Through Grid Search : %.3f'%multinomial_nb_grid.best_score_)
print('Best Parameters : ',multinomial_nb_grid.best_params_)
print(classification_report(label_test,y_predict))

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Accuracy Through Grid Search : 0.738
Best Parameters :  {'alpha': 10}


In [ ]:
## DT

In [130]:
nb_tfidf = DecisionTreeClassifier(criterion="entropy")
nb_tfidf.fit(Xlr_train, label_train)  #model
y_predict = nb_tfidf.predict(Xlr_test)
print(classification_report(label_test,y_predict))


              precision    recall  f1-score   support

           0       0.67      0.68      0.68       277
           1       0.76      0.73      0.74       301
           2       0.83      0.85      0.84       275

    accuracy                           0.75       853
   macro avg       0.75      0.76      0.76       853
weighted avg       0.75      0.75      0.75       853



In [ ]:
 ## SVC

In [111]:
nb_tfidf = svm.SVC(kernel='rbf', C=1,gamma='auto')
nb_tfidf.fit(Xlr_train, label_train)  #modelb
y_predict = nb_tfidf.predict(Xlr_test)
print(classification_report(label_test,y_predict))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       296
           1       0.00      0.00      0.00       284
           2       0.32      1.00      0.48       273

    accuracy                           0.32       853
   macro avg       0.11      0.33      0.16       853
weighted avg       0.10      0.32      0.16       853



C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [2]:
## SVC with tf-idf

In [ ]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(train.tweet).toarray()
labels = train.label
features.shape
model = LinearSVC()
model.fit(features, labels)
features_test = tfidf.transform(test.tweet)
features_test.shape
pred = model.predict(features_test)
submission=test
submission['label']=pred
# submission.drop(['', 'Phrase'], axis=1, inplace=True)
# submission.to_csv("submission.csv", index = False)
submission.to_csv("D:/res3.csv")

In [ ]:
## GB

In [131]:
gboost_m = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1)
train_pred = gboost_m.fit(Xlr_train, label_train).predict(Xlr_test)
print(classification_report(label_test,y_predict))


              precision    recall  f1-score   support

           0       0.67      0.68      0.68       277
           1       0.76      0.73      0.74       301
           2       0.83      0.85      0.84       275

    accuracy                           0.75       853
   macro avg       0.75      0.76      0.76       853
weighted avg       0.75      0.75      0.75       853



In [ ]:
sequence_length = 32
max_features = 20000 # this is the number of words we care about

tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>')
tokenizer.fit_on_texts(data['tweet'].values)

# this takes our sentences and replaces each word with an integer
X = tokenizer.texts_to_sequences(data['tweet'].values)
x_test = tokenizer.texts_to_sequences(test['tweet'].values)

# we then pad the sequences so they're all the same length (sequence_length)
X = pad_sequences(X, sequence_length)
x_test = pad_sequences(x_test, sequence_length)
y = pd.get_dummies(data['label']).values

# where there isn't a test set, Kim keeps back 10% of the data for testing, I'm going to do the same since we have an ok amount to play with
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print("test set size " + str(len(X_test)))

In [ ]:
# Building the BASELINE MODEL
base_model = Sequential()
base_model.add(Embedding(num_words,128,input_length=X_train.shape[1]))
base_model.add(GlobalAveragePooling1D())
base_model.add(Dense(8,activation='relu'))
base_model.add(Dense(3,activation='softmax'))
base_model.summary()
base_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history_base = base_model.fit(X_train, y_train ,epochs=20, validation_split=0.2)
y_hat_5 = base_model.predict(X_test)
loss, accuracy = base_model.evaluate(X_test, y_test, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))
predictions = base_model.predict(x_test)
y_prob = base_model.predict(x_test)
y_classes = y_prob.argmax(axis=-1) 
test['label'] = y_classes
test.to_csv("D:/cnn3.csv")

In [ ]:
#Model 1: Random embeddings

In [56]:
embedding_dim = 300 # Kim uses 300 here
num_filters = 100

inputs = Input(shape=(sequence_length,), dtype='int32')

# use a random embedding for the text
embedding_layer = Embedding(input_dim=max_features, output_dim=embedding_dim, input_length=sequence_length)(inputs)

reshape = Reshape((sequence_length, embedding_dim, 1))(embedding_layer)

# Note the relu activation which Kim specifically mentions
# He also uses an l2 constraint of 3
# Also, note that the convolution window acts on the whole 200 dimensions - that's important
conv_0 = Conv2D(num_filters, kernel_size=(3, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(4, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(5, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape)

# perform max pooling on each of the convoluations
maxpool_0 = MaxPool2D(pool_size=(sequence_length - 3 + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(sequence_length - 4 + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(sequence_length - 5 + 1, 1), strides=(1,1), padding='valid')(conv_2)

# concat and flatten
concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)

# do dropout and predict
dropout = Dropout(0.5)(flatten)
output = Dense(units=3, activation='softmax')(dropout)

model = Model(inputs=inputs, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

batch_size = 32 # Kim uses 50 here, I have a slightly smaller sample size than num
history = model.fit(X_train, y_train, epochs=20, batch_size=batch_size, verbose=1, validation_split=0.1, shuffle=True)

In [ ]:
tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>')
tokenizer.fit_on_texts(test['tweet'].values)
# this takes our sentences and replaces each word with an integer
X = tokenizer.texts_to_sequences(test['tweet'].values)
# we then pad the sequences so they're all the same length (sequence_length)
X = pad_sequences(X, sequence_length)

predictions = model.predict(X)
y_prob = model.predict(X)
y_classes = y_prob.argmax(axis=-1) 
test['label'] = y_classes
test.to_csv("D:/cnn.csv")

In [ ]:
y_hat = model.predict(X_test)
accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat)))
confusion_matrix(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat)))
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

In [ ]:
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')
# plt.show()

# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')
# plt.show()

In [32]:
#Model 2: Static word2vec

In [13]:
embeddings_index = {}
f = open(os.path.join('D:/', 'glove.42B.300d.txt'),encoding='utf-8') #glove.42B.300d.txt
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

embedding_dim = 300 # Kim uses 300 here
num_filters = 100
num_words = min(max_features, len(word_index)) + 1
print(num_words)

# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i > max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)

Found 1917494 word vectors.


In [16]:
inputs_2 = Input(shape=(sequence_length,), dtype='int32')

# note the `trainable=False`, later we will make this layer trainable
embedding_layer_2 = Embedding(num_words,
                            embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=sequence_length,
                            trainable=False)(inputs_2)

reshape_2 = Reshape((sequence_length, embedding_dim, 1))(embedding_layer_2)

conv_0_2 = Conv2D(num_filters, kernel_size=(3, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_2)
conv_1_2 = Conv2D(num_filters, kernel_size=(4, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_2)
conv_2_2 = Conv2D(num_filters, kernel_size=(5, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_2)

maxpool_0_2 = MaxPool2D(pool_size=(sequence_length - 3 + 1, 1), strides=(1,1), padding='valid')(conv_0_2)
maxpool_1_2 = MaxPool2D(pool_size=(sequence_length - 4 + 1, 1), strides=(1,1), padding='valid')(conv_1_2)
maxpool_2_2 = MaxPool2D(pool_size=(sequence_length - 5 + 1, 1), strides=(1,1), padding='valid')(conv_2_2)

concatenated_tensor_2 = Concatenate(axis=1)([maxpool_0_2, maxpool_1_2, maxpool_2_2])
flatten_2 = Flatten()(concatenated_tensor_2)

dropout_2 = Dropout(0.5)(flatten_2)
output_2 = Dense(units=3, activation='softmax')(dropout_2)

model_2 = Model(inputs=inputs_2, outputs=output_2)
model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_2.summary())

batch_size = 32
history_2 = model_2.fit(X_train, y_train, epochs=20, batch_size=batch_size, verbose=1, validation_split=0.2)

In [ ]:
y_hat_2 = model_2.predict(X_test)
accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat_2)))
confusion_matrix(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat_2)))
loss, accuracy = model_2.evaluate(X_test, y_test, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

In [ ]:
predictions = model_2.predict(x_test)
y_prob = model_2.predict(x_test)
y_classes = y_prob.argmax(axis=-1) 
test['label'] = y_classes
test.to_csv("D:/cnn1.csv")

In [143]:
# plt.plot(history_2.history['accuracy'])
# plt.plot(history_2.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')
# plt.show()

# plt.plot(history_2.history['loss'])
# plt.plot(history_2.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')
# plt.show()

In [41]:
#Model 3: w2v with trainable embeddings

In [20]:
inputs_3 = Input(shape=(sequence_length,), dtype='int32')
embedding_layer_3 = Embedding(num_words,
                            embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=sequence_length,
                            trainable=True)(inputs_3)

reshape_3 = Reshape((sequence_length, embedding_dim, 1))(embedding_layer_3)

# note the relu activation
conv_0_3 = Conv2D(num_filters, kernel_size=(3, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_3)
conv_1_3 = Conv2D(num_filters, kernel_size=(4, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_3)
conv_2_3 = Conv2D(num_filters, kernel_size=(5, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_3)

maxpool_0_3 = MaxPool2D(pool_size=(sequence_length - 3 + 1, 1), strides=(1,1), padding='valid')(conv_0_3)
maxpool_1_3 = MaxPool2D(pool_size=(sequence_length - 4 + 1, 1), strides=(1,1), padding='valid')(conv_1_3)
maxpool_2_3 = MaxPool2D(pool_size=(sequence_length - 5 + 1, 1), strides=(1,1), padding='valid')(conv_2_3)

concatenated_tensor_3 = Concatenate(axis=1)([maxpool_0_3, maxpool_1_3, maxpool_2_3])
flatten_3 = Flatten()(concatenated_tensor_3)

dropout_3 = Dropout(0.5)(flatten_3)
output_3 = Dense(units=3, activation='softmax')(dropout_3)

model_3 = Model(inputs=inputs_3, outputs=output_3)
model_3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_3.summary())

batch_size = 32
history_3 = model_3.fit(X_train, y_train, epochs=20, batch_size=batch_size, verbose=1, validation_split=0.2)

In [ ]:
y_hat_3 = model_3.predict(X_test)
accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat_3)))
confusion_matrix(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat_3)))
loss, accuracy = model_3.evaluate(X_test, y_test, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

In [ ]:
predictions = model_3.predict(x_test)
y_prob = model_3.predict(x_test)
y_classes = y_prob.argmax(axis=-1) 
test['label'] = y_classes
test.to_csv("D:/cnn2.csv")

In [149]:
# plt.plot(history_3.history['accuracy'])
# plt.plot(history_3.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')
# plt.show()

# plt.plot(history_3.history['loss'])
# plt.plot(history_3.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')
# plt.show()

In [ ]:
print("CNN random       : " + str(accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat)))))
print("CNN static       : " + str(accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat_2)))))
print("CNN trainable    : " + str(accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat_3)))))
# LR = 79
# NB = 75
# DT = 73

In [21]:
# Removing overfitiing
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
embedding_dim = 300 # Kim uses 300 here
num_filters = 100
num_words = min(max_features, len(word_index)) + 1
regularise = tf.keras.regularizers.l2(0.001)

model_r = Sequential()
model_r.add(Embedding(num_words,128,input_length=X_train.shape[1]))
model_r.add(Dropout(0.5))
model_r.add(GlobalAveragePooling1D())
model_r.add(Dense(8,activation='relu',kernel_regularizer=regularise))
model_r.add(Dropout(0.5))
model_r.add(Dense(3,activation='softmax'))
model_r.summary()

#Compiling the model
model_r.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history_r = model_r.fit(X_train, y_train ,epochs=20, validation_split=0.2)
y_hat_6 = model_r.predict(X_test)

loss, accuracy = model_r.evaluate(X_test, y_test, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

predictions = model_r.predict(x_test)
y_prob = model_r.predict(x_test)
y_classes = y_prob.argmax(axis=-1) 
test['label'] = y_classes
test.to_csv("D:/cnn4.csv")

Found 10398 unique tokens.
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 128)           1331072   
                                                                 
 dropout (Dropout)           (None, 32, 128)           0         
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 8)                 1032      
                                                                 
 dropout_1 (Dropout)         (None, 8)                 0         
                                                                 
 dense_1 (Dense)             (None, 3)                 27        
                           

In [70]:
# LSTM

In [32]:
#Building the model
from keras.constraints import max_norm
model1 = Sequential([
    Embedding(num_words,128,input_length=X_train.shape[1]),
    Dropout(0.5),
    LSTM(32,kernel_constraint=max_norm(3)),
    Dense(32,activation='relu',kernel_regularizer=regularise),
    Dropout(0.5),
    Dense(3,activation='softmax')
])
#Compiling the model
model1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#Fitting the model
history1 =  model1.fit(X_train, y_train ,epochs=20, validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',patience=3)])
y_hat_7 = model1.predict(X_test)

loss, accuracy = model1.evaluate(X_test, y_test, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))
predictions = model1.predict(x_test)
y_prob = model1.predict(x_test)
y_classes = y_prob.argmax(axis=-1) 
test['label'] = y_classes
test.to_csv("D:/cnn5.csv")

Epoch 1/20
77/77 [==============================] - 4s 26ms/step - loss: 1.1038 - accuracy: 0.4395 - val_loss: 1.0286 - val_accuracy: 0.5961
Epoch 2/20
77/77 [==============================] - 1s 19ms/step - loss: 0.8073 - accuracy: 0.6497 - val_loss: 0.7595 - val_accuracy: 0.6906
Epoch 3/20
77/77 [==============================] - 1s 19ms/step - loss: 0.4939 - accuracy: 0.8102 - val_loss: 0.7252 - val_accuracy: 0.7524
Epoch 4/20
77/77 [==============================] - 1s 19ms/step - loss: 0.3238 - accuracy: 0.8957 - val_loss: 0.8476 - val_accuracy: 0.6971
Epoch 5/20
77/77 [==============================] - 2s 20ms/step - loss: 0.1909 - accuracy: 0.9479 - val_loss: 0.9519 - val_accuracy: 0.7150
Epoch 6/20
11/11 [==============================] - 0s 5ms/step


In [ ]:
#tuning the model

In [22]:
#Building the model
model2 = Sequential([
    Embedding(num_words,128,input_length=X_train.shape[1]),
    Bidirectional(LSTM(128,return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(32,activation='relu'),
    Dropout(0.5),
    Dense(3,activation='softmax')
])
#Compiling the model
model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#Fitting the model
history2 =  model2.fit(X_train, y_train ,epochs=20, validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',patience=3)])
y_hat_8 = model2.predict(X_test)
loss, accuracy = model2.evaluate(X_test, y_test, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))
predictions = model2.predict(x_test)
y_prob = model2.predict(x_test)
y_classes = y_prob.argmax(axis=-1) 
test['label'] = y_classes
test.to_csv("D:/cnn6.csv")

Epoch 1/20
77/77 [==============================] - 13s 80ms/step - loss: 1.0574 - accuracy: 0.4033 - val_loss: 0.9574 - val_accuracy: 0.6124
Epoch 2/20
77/77 [==============================] - 4s 56ms/step - loss: 0.6438 - accuracy: 0.7426 - val_loss: 0.6321 - val_accuracy: 0.7443
Epoch 3/20
77/77 [==============================] - 4s 57ms/step - loss: 0.3301 - accuracy: 0.9018 - val_loss: 0.7588 - val_accuracy: 0.7313
Epoch 4/20
77/77 [==============================] - 4s 57ms/step - loss: 0.1588 - accuracy: 0.9597 - val_loss: 1.1014 - val_accuracy: 0.7166
Epoch 5/20
11/11 [==============================] - 2s 15ms/step
